In [130]:
import logging.config
import pickle
from pathlib import Path
from pprint import pprint
import sys
import numpy as np
import pandas as pd
from dateutil.parser import parse
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from tqdm.autonotebook import tqdm
sys.path.append('..')
from IPython.display import display
import random
import math
from tqdm import tqdm

from src.nuforc import SETTINGS
from src.nuforc.utility import *
from src.nuforc.geocoding.geocoder import Geolocator
from src.nuforc.geocoding.wrangling import replace_unparsed_with_none, join_columns
from datetime import date
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
from geopandas import GeoSeries
from scipy.stats import mode

#### Data loading

In [136]:
Path(r'.\\data\\nuforc_events.csv')

WindowsPath('data/nuforc_events.csv')

In [ ]:
# Events
raw = pd.read_csv(Path(r'..\\data\\nuforc_events.csv'))
raw = replace_unparsed_with_none(raw)
raw['address'] = raw.apply(lambda row: join_columns(row['city'], row['state'], row['country']), axis=1)

# Coords
coords = pd.read_csv(Path(r'..\\gis\\csv\\geolocated_addresses.csv'), names=['address', 'latitude', 'longitude'])

In [141]:
# Zipcode polygons
polygons = gpd.read_file(Path(r'..\\gis\\shp\\cb_2018_us_zcta510_500k.shp'))

In [188]:
# Define a function to lowercase strings while handling None values
def lowercase_string(s):
    if isinstance(s, str):
        return s.lower()
    else:
        return s

df = raw.copy()
df = pd.merge(left=df, right=coords, on='address')
df['shape'] = df['shape'].apply(lambda x: lowercase_string(x))

In [189]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf.crs = polygons.crs
gdf = gpd.sjoin(gdf, polygons, how='left', op='within')

C:\Users\Chimney\AppData\Local\pypoetry\Cache\virtualenvs\nuforc-Wvo9mW45-py3.8\lib\site-packages\IPython\core\interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [192]:
# Assuming you have a DataFrame named 'df' with columns 'ZCTA5CE10' and 'shape'

# Group by "ZCTA5CE10" and apply a lambda function to find the mode of "shape"
most_frequent_shape = gdf.groupby('ZCTA5CE10')['shape'].apply(lambda x: mode(x)[0][0] if not x.empty else "N/A")

# Set the index of the result DataFrame
most_frequent_shape.index.name = 'ZCTA5CE10'
most_frequent_shape = most_frequent_shape.reset_index()

# Rename the columns for better clarity
most_frequent_shape.columns = ['ZCTA5CE10', 'most_frequent_shape']
most_frequent_shape.to_csv(f"I:\\Dropbox\\Python\\nuforc\\gis\\csv\\most_frequent_shape.csv")

C:\Users\Chimney\AppData\Local\Temp\ipykernel_8840\2932139974.py:4: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  most_frequent_shape = gdf.groupby('ZCTA5CE10')['shape'].apply(lambda x: mode(x)[0][0] if not x.empty else "N/A")
C:\Users\Chimney\AppData\Local\pypoetry\Cache\virtualenvs\nuforc-Wvo9mW45-py3.8\lib\site-packages\scipy\stats\_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Chimney\AppData\Local\Temp\ipykernel_8840\2932139974.py:4: DeprecationWarning: Support for non-numeric ar

In [193]:
random_points = []
# Generate random points within each polygon and add them to the new GeoDataFrame

for idx, row in tqdm(gdf.iterrows()):
    polygon = polygons[polygons['ZCTA5CE10'] == row['ZCTA5CE10']]
    s = gpd.GeoSeries(polygon['geometry'])
    
    random_point = s.sample_points(size=1)
    try:
        row['randomized_x'] = random_point.x.iloc[0]
        row['randomized_y'] = random_point.y.iloc[0]
    except:
        continue
    random_points.append(row)

141380it [21:50, 107.92it/s]


In [195]:
randomized_locations_df = pd.DataFrame(random_points)

In [201]:
randomized_locations_df[['duration', 'occurred_time', 'description', 'shape', 'address', 'ZCTA5CE10', 'latitude', 'longitude', 'randomized_x', 'randomized_y']].to_csv(f"I:\\Dropbox\\Python\\nuforc\\gis\\csv\\randomized_locations.csv")